<a href="https://colab.research.google.com/github/AlisonJD/tb_examples/blob/main/Publish_SQL_based_endpoints_on_NGINX_log_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Publish SQL-based endpoints on NGINX log analysis

Based on Tinybird blog post:

https://blog.tinybird.co/2021/01/28/nginx-log-analysis/

If you have opened the notebook in Google Colab then `Copy to Drive` (see above).

In [2]:
#@title Mount your Google Drive to save and use local files
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

% cd "/content/gdrive/My Drive/Colab Notebooks/Tinybird/tb_examples"

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/Tinybird/tb_examples


In [3]:
#@title Install Tinybird CLI, utilities and your token
!pip install tinybird-cli -q
!sudo apt-get install jq

import os
import re

if not os.path.isfile('.tinyb'):
  !tb auth

if not os.path.isdir('datasources'):
  !tb init

     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 81 kB 10.8 MB/s 
     |████████████████████████████████| 46 kB 4.1 MB/s 
     |████████████████████████████████| 61 kB 8.1 MB/s 
     |████████████████████████████████| 54 kB 2.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.25.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libjq1 

In [4]:
#@title Helper function to write to files
def write_text_to_file(filename, text):
  with open(filename, 'w') as f: f.write(text)

# Worked Example from Blog: 
##Publish SQL-based endpoints on NGINX log analysis

Tinybird can be used to analyze datasets like logs.

Here we use Tinybird to:
 - analyze NGINX logs
 - publish SQL queries as API endpoints

We also show you how to model your Data Sources and API Endpoints to make them faster.


## 1. Build the Data Source from a Sample NGINX Log
Ingest the CSV and transform the columns.

In [5]:
!tb datasource generate https://raw.githubusercontent.com/tinybirdco/log_parsing_template/main/access.log.csv --force

** Generated datasources/access_log.datasource
** => Create it on the server running: $ tb push datasources/access_log.datasource
** => Append data using: $ tb datasource append access.log https://raw.githubusercontent.com/tinybirdco/log_parsing_template/main/access.log.csv`



In [6]:
!tb datasource rm access_log --yes

** Data Source 'access_log' deleted


In [7]:
!tb push datasources/access_log.datasource --force

** Processing datasources/access_log.datasource
** Building dependencies
** Running access_log 
** 'access_log' created
** Not pushing fixtures


In [8]:
!tb datasource append access_log 'https://raw.githubusercontent.com/tinybirdco/log_parsing_template/main/access.log.csv'

** 🥚 starting import process
** 🐥 done
** Total rows in access_log: 67351
** Data appended to Data Source 'access_log' successfully!
** Data pushed to access_log


Looking at a single record, we see that we need to add column names and extract information from columns.

In [9]:
!tb sql "select * from access_log limit 1"

---------------------------------------------------------------------
column_00: 10.86.160.14
column_01: -
column_02: -
column_03: [19/Jan/2021:06:25:37
column_04: +0000]
column_05: GET /v0/pipes/pro__ct___v0.json?id_c=18&token=XYZ HTTP/1.1
column_06: 200
column_07: 455
column_08: -
column_09: Apache-HttpClient/4.5.10 (Java/11.0.8)
---------------------------------------------------------------------


Let's do that with a Pipe:

In [10]:
filename="pipes/access_log_transform.pipe"
text='''
DESCRIPTION extract column data from raw access log and name columns
NODE extract_column_data
SQL >
    select
        IPv4StringToNum(column_00) as ip,
        parseDateTimeBestEffort(replaceOne(substring(column_03, 2), ':', ' ')) as time,
        splitByChar(' ', column_05) as tt,
        tt[1] as method,
        tt[2] as path,
        tt[3] as protocol,
        column_06 as status_code,
        column_07 as bytes,
        column_09 as user_agent
    from access_log
'''

write_text_to_file(filename, text)

!tb push pipes/access_log_transform.pipe --force

** Processing pipes/access_log_transform.pipe
** Building dependencies
** Running access_log_transform 
** => Test endpoint at https://api.tinybird.co/v0/pipes/access_log_transform.json
** 'access_log_transform' created
** Not pushing fixtures


In [11]:
!tb sql "select * from access_log_transform limit 1"

---------------------------------
ip: 1382418444
time: 2021-01-19 11:48:24
tt: ['GET', '/', 'HTTP/1.1']
method: GET
path: /
protocol: HTTP/1.1
status_code: 301
bytes: 194
user_agent: Mozilla/5.0 zgrab/0.x
---------------------------------


## 2. Publish an Endpoint
Create an Endpoint for the number of requests and average bytes for each IP address. An Endpoint can be consumed by your data products.

In [12]:
filename="pipes/requests_per_endpoint.pipe"
text='''
DESCRIPTION requests per endpoint
NODE grouping
SQL >
    %
    SELECT
        ip,
        count() AS request_count,
        avg(bytes) as avg_bytes
    FROM access_log_transform
    GROUP BY ip
    ORDER BY request_count DESC

NODE endpoint
SQL >
    select IPv4NumToString(ip) as ip_address,
           request_count,
           avg_bytes
    from grouping
'''

write_text_to_file(filename, text)

!tb push pipes/requests_per_endpoint.pipe --force --no-check

** Processing pipes/requests_per_endpoint.pipe
** Building dependencies
** Running requests_per_endpoint 
** => Test endpoint at https://api.tinybird.co/v0/pipes/requests_per_endpoint.json
** 'requests_per_endpoint' created
** Not pushing fixtures


In [13]:
TOKEN = !cat ../.tinyb | jq .token 
TOKEN = re.search(r'\"(.*?)\"', TOKEN[0]).group()[1:-1]

In [14]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint.json?token=$TOKEN

{
	"meta":
	[
		{
			"name": "ip_address",
			"type": "String"
		},
		{
			"name": "request_count",
			"type": "UInt64"
		},
		{
			"name": "avg_bytes",
			"type": "Float64"
		}
	],

	"data":
	[
		{
			"ip_address": "10.86.160.21",
			"request_count": 39624,
			"avg_bytes": 438.6311326468807
		},
		{
			"ip_address": "10.86.160.20",
			"request_count": 21376,
			"avg_bytes": 438.7071014221557
		},
		{
			"ip_address": "10.86.160.24",
			"request_count": 646,
			"avg_bytes": 746.1099071207431
		},
		{
			"ip_address": "10.86.160.25",
			"request_count": 616,
			"avg_bytes": 1068.237012987013
		},
		{
			"ip_address": "10.86.160.15",
			"request_count": 612,
			"avg_bytes": 867.1470588235294
		},
		{
			"ip_address": "10.86.160.26",
			"request_count": 606,
			"avg_bytes": 864.1518151815181
		},
		{
			"ip_address": "10.86.160.28",
			"request_count": 604,
			"avg_bytes": 594.9105960264901
		},
		{
			"ip_address": "10.86.160.27",
			"request_count": 579,
			"avg_bytes": 1031.90155440414

In [15]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint.json?token=$TOKEN |head -n 24

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0{
	"meta":
	[
		{
			"name": "ip_address",
			"type": "String"
		},
		{
			"name": "request_count",
			"type": "UInt64"
		},
		{
			"name": "avg_bytes",
			"type": "Float64"
		}
	],

	"data":
	[
		{
			"ip_address": "10.86.160.21",
			"request_count": 39624,
			"avg_bytes": 438.6311326468807
		},
100  6631  100  6631    0     0  16829      0 --:--:-- --:--:-- --:--:-- 16829
(23) Failed writing body


In [16]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint.json?token=$TOKEN |tail -n 16

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6630  100  6630    0     0  17447      0 --:--:-- --:--:-- --:--:-- 17401
		{
			"ip_address": "209.141.60.10",
			"request_count": 1,
			"avg_bytes": 264
		}
	],

	"rows": 71,

	"statistics":
	{
		"elapsed": 0.00295311,
		"rows_read": 67351,
		"bytes_read": 1686164
	}
}


In [17]:
!tb sql "SELECT count() FROM requests_per_endpoint"

-----------
| count() |
-----------
|      71 |
-----------


In [18]:
!tb sql "SELECT uniqExact(ip) from access_log_transform"

-----------------
| uniqExact(ip) |
-----------------
|            71 |
-----------------


## 3. Create a Real-Time Endpoint using a Materialized View
For Endpoints serving real-time dashboards, where the user can pick, for example, different date ranges or add filters, it is unacceptable to have to wait. If we had millions of entries a day, the generated endpoints wouldn’t be as fast as needed.

The solution is to use a materialized view.

In [19]:
filename="pipes/requests_per_endpoint_mv.pipe"
text='''
DESCRIPTION materialized view
NODE matview
SQL >
    SELECT
        ip,
        countState() AS request_count,
        avgState(bytes) as avg_bytes
    FROM access_log_transform
    GROUP BY ip

TYPE Materialized
DATASOURCE requests_per_endpoint_ds
'''

write_text_to_file(filename, text)

filename="datasources/requests_per_endpoint_ds.datasource"
text='''
DESCRIPTION materialized view
SCHEMA >
    ip UInt32,
    request_count AggregateFunction(count),
    avg_bytes AggregateFunction(avg, Int32)

ENGINE AggregatingMergeTree
ENGINE_SORTING_KEY ip
'''

write_text_to_file(filename, text)

In [20]:
!tb pipe rm requests_per_endpoint_mv --yes
!tb datasource rm requests_per_endpoint_ds --yes

** Pipe 'requests_per_endpoint_mv' deleted
** Data Source 'requests_per_endpoint_ds' deleted


- The materialized view Pipe uses `countState` and `avgState` for the intermediate states. The Data Source has `AggregateFunction` data types to store those intermediate states.
- The Engine is not the regular `MergeTree` but instead an `AggregatingMergeTree` that tells ClickHouse to aggregate columns on merge operations.
- The sorting key tells ClickHouse which column is used for grouping.

In [21]:
!tb push datasources/requests_per_endpoint_ds.datasource

** Processing datasources/requests_per_endpoint_ds.datasource
** Building dependencies
** Running requests_per_endpoint_ds 
** 'requests_per_endpoint_ds' created
** Not pushing fixtures


In [22]:
!tb push pipes/requests_per_endpoint_mv.pipe --populate --force

** Processing pipes/requests_per_endpoint_mv.pipe
** Building dependencies
** Running requests_per_endpoint_mv 
** Materialized node 'matview' using the Data Source 'requests_per_endpoint_ds'
** Populating job url https://api.tinybird.co/v0/jobs/22953f18-30d5-4753-b219-ba1a3be979a4
** 'requests_per_endpoint_mv' created
** Not pushing fixtures



`--populate` loads the materialized view with the data already in `access_log_transform`

In [23]:
!tb sql "select uniqExact(ip) from requests_per_endpoint_ds" --stats

** Query took 0.000365167 seconds
** Rows read: 23
** Bytes read: 92 bytes
-----------------
| uniqExact(ip) |
-----------------
|            23 |
-----------------


If we push new data with new IP addresses, the number of rows will rise but if we compare this to the number of rows in the original table, working with this view will be much faster.

Note that this view is updated when you append new data, it does not need to recalculate with all the data - thanks to the intermediate states.

In [24]:
filename="pipes/requests_per_endpoint_fast.pipe"
text='''
DESCRIPTION requests per endpoint fast
NODE grouping
SQL >
    %
    SELECT
        ip,
        countMerge(request_count) AS request_count,
        avgMerge(avg_bytes) as avg_bytes
    FROM requests_per_endpoint_ds
    GROUP BY ip
    ORDER BY request_count DESC

NODE endpoint
SQL >
    select IPv4NumToString(ip) as ip_address,
           request_count,
           avg_bytes
    from grouping
'''

write_text_to_file(filename, text)

!tb push pipes/requests_per_endpoint_fast.pipe --force --no-check

** Processing pipes/requests_per_endpoint_fast.pipe
** Building dependencies
** Running requests_per_endpoint_fast 
** => Test endpoint at https://api.tinybird.co/v0/pipes/requests_per_endpoint_fast.json
** 'requests_per_endpoint_fast' created
** Not pushing fixtures


In [25]:
TOKEN = !cat .tinyb | jq .token 
TOKEN = re.search(r'\"(.*?)\"', TOKEN[0]).group()[1:-1]

In [26]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint.json\?token\=$TOKEN | jq .statistics

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  6631  100  6631    0     0  16872      0 --:--:-- --:--:-- --:--:-- 16872
{
  "elapsed": 0.002935396,
  "rows_read": 67351,
  "bytes_read": 1686164
}


In [27]:
!curl https://api.tinybird.co/v0/pipes/requests_per_endpoint_fast.json\?token\=$TOKEN | jq .statistics

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2464  100  2464    0     0   6623      0 --:--:-- --:--:-- --:--:--  6605
{
  "elapsed": 0.000518512,
  "rows_read": 23,
  "bytes_read": 8652
}


The new API Endpoint using the materialized view is faster and reads far less data.

Materialized views could be used for:

- unique ip addresses by day: 
```
select toDate(time) day, uniqState(ip_address) uniq_ip 
from access_log 
group by day
```
- percentile 95 of payload size per hour:
```
select toStartOfHour(time) hour, quantileState(0.95)(ip_address) q95
from access_log_transform 
group by hour
```
- requests per month: 
```
select toStartOfMonth(time) month, countState() requests_count 
from access_log_transform 
group by month
```

